In [ ]:
!wget https://github.com/AndreyK2001/BERT-Classification/blob/main/data_ru.txt

--2023-12-01 15:10:16--  https://github.com/AndreyK2001/BERT-Classification/blob/main/data.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4185 (4.1K) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   4.09K  --.-KB/s    in 0s      

2023-12-01 15:10:16 (68.5 MB/s) - ‘data.csv’ saved [4185/4185]



In [1]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
import pandas as pd
import numpy as np

df = pd.read_csv('./data_ru.txt')
df.head()

,text,class
0,\nВВС Франции провели испытания перспективной ...,Оружие
1,\nИсследователи из Цюрихского университета нау...,Технологии
2,\nОрбитальная обсерватория «Хаббл» получил нов...,Технологии
3,\nАмериканская компания Ford начала снежные ис...,Транспорт
4,\nАмериканская компания Zipline и правительств...,Гаджеты


In [2]:
df['class'].value_counts()

Наука         3506
Технологии    1452
Оружие        1279
Космос         470
Гаджеты        445
Транспорт      342
Name: class, dtype: int64

In [3]:
possible_labels = df['class'].unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Оружие': 0,
 'Технологии': 1,
 'Транспорт': 2,
 'Гаджеты': 3,
 'Наука': 4,
 'Космос': 5}

In [4]:
df['label'] = df['class'].replace(label_dict)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                  df.label.values,
                                                  test_size=0.15,
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['class', 'label', 'data_type']).count()

text
class      label data_type      
Гаджеты    3     train       378
                 val          67
Космос     5     train       399
                 val          71
Наука      4     train      2980
                 val         526
Оружие     0     train      1087
                 val         192
Технологии 1     train      1234
                 val         218
Транспорт  2     train       291
                 val          51

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased',
                                          do_lower_case=True)

encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [7]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train,
                              sampler=RandomSampler(dataset_train),
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val,
                                   sampler=SequentialSampler(dataset_val),
                                   batch_size=batch_size)

In [9]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5,
                  eps=1e-8)

epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        tqdm.write(f'Class: {label_dict_inverse[label]}')
        tqdm.write(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')


In [11]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = 'cuda'

model = model.to(device)

def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

for epoch in tqdm(range(1, epochs+1)):

    model.train()

    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        outputs = model(**inputs)

        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})



    tqdm.write(f'\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    accuracy_per_class(predictions, true_vals)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/2123 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.7711231450035245
Validation loss: 0.784049636181444
F1 Score (Weighted): 0.790260653953359
Class: Оружие
Accuracy: 181/192

Class: Технологии
Accuracy: 164/218

Class: Транспорт
Accuracy: 33/51

Class: Гаджеты
Accuracy: 23/67

Class: Наука
Accuracy: 456/526

Class: Космос
Accuracy: 36/71



Epoch 2:   0%|          | 0/2123 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.6217487968173881
Validation loss: 0.7442660377807915
F1 Score (Weighted): 0.8131774063130771
Class: Оружие
Accuracy: 174/192

Class: Технологии
Accuracy: 142/218

Class: Транспорт
Accuracy: 37/51

Class: Гаджеты
Accuracy: 39/67

Class: Наука
Accuracy: 483/526

Class: Космос
Accuracy: 41/71



...


Epoch 3
Training loss: 0.5119745404278251
Validation loss: 0.8717783904528866
F1 Score (Weighted): 0.8168562716741167
Class: Оружие
Accuracy: 178/192

Class: Технологии
Accuracy: 152/218

Class: Транспорт
Accuracy: 33/51

Class: Гаджеты
Accuracy: 43/67

Class: Наука
Accuracy: 478/526

Class: Космос
Accuracy: 36/71



Epoch 4:   0%|          | 0/2123 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.3985724191577985
Validation loss: 0.8798239802541211
F1 Score (Weighted): 0.8238427290987521
Class: Оружие
Accuracy: 174/192

Class: Технологии
Accuracy: 156/218

Class: Транспорт
Accuracy: 32/51

Class: Гаджеты
Accuracy: 38/67

Class: Наука
Accuracy: 486/526

Class: Космос
Accuracy: 42/71



Epoch 5:   0%|          | 0/2123 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.28519173586753754
Validation loss: 0.9994179185770917
F1 Score (Weighted): 0.831191808954647
Class: Оружие
Accuracy: 180/192

Class: Технологии
Accuracy: 160/218

Class: Транспорт
Accuracy: 35/51

Class: Гаджеты
Accuracy: 38/67

Class: Наука
Accuracy: 479/526

Class: Космос
Accuracy: 44/71



In [12]:
torch.save(model.state_dict(), './ru_classsifier.model')